In [90]:
import pandas as pd
import numpy as np
import pickle

In [91]:
with (open("chgk/results.pkl", "rb")) as openfile:
    results = pickle.load(openfile)
with (open("chgk/tournaments.pkl", "rb")) as openfile:
    tournaments = pickle.load(openfile)
with (open("chgk/players.pkl", "rb")) as openfile:
    players = pickle.load(openfile)

In [92]:
train_tournaments = {}
test_tournaments = {}
for key, value in tournaments.items():
    if len(results[key]) and 'mask' in results[key][0]:
        if results[key][0]['mask'] != None: 
            if int(value['dateStart'].split('-')[0]) == 2019:
                train_tournaments[key] = value
            if int(value['dateStart'].split('-')[0]) == 2020:
                test_tournaments[key] = value

Считаем веса вопросов

In [132]:
weights = {}
for idx, data in train_tournaments.items():
    totalQs = np.sum(list(data['questionQty'].values()))
    masks = []
    for team in results[idx]:
        if team['mask'] and len(team['mask']) == totalQs and 'X' not in team['mask'] and '?' not in team['mask']:
            masks.append(list(map(int, team['mask'])))
    weights[idx] = 1 - np.mean(masks, axis=0)   
    

/opt/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Создаем таблицу игроков и вопросов

In [97]:
df = []
for idx, data in train_tournaments.items():
    totalQs = np.sum(list(data['questionQty'].values()))
    
    for team in results[idx]:
        if team['mask'] and len(team['mask']) == totalQs and 'X' not in team['mask'] and '?' not in team['mask']:
            mask = list(map(int, team['mask']))
            for member in team["teamMembers"]:
                for iq in range(totalQs):
                    df.append((member["player"]["id"],
                               weights[idx][iq],
                               mask[iq],
                               f"{idx}_"+str(iq))) 
df = pd.DataFrame(df, columns=["player", "difficult", "answer", "question_id"])

In [99]:
df.head()


,player,difficult,answer,question_id
0,6212,0.116883,1,4772_0
1,6212,0.220779,1,4772_1
2,6212,0.554113,1,4772_2
3,6212,0.480519,1,4772_3
4,6212,0.121212,1,4772_4


## 2 часть

In [ ]:
from sklearn.linear_model import LogisticRegression
player_rating = {}
all_players = df["player"].unique()
model = LogisticRegression(C=100, solver="lbfgs")
for player in all_players:
    train = df[df["player"]==player]
    data_X = train[["difficult"]]
    data_y = train["answer"]
    if np.unique(data_y).shape[0] > 1:
        model.fit(data_X, data_y)
        predict = model.predict_proba(np.arange(0, 1, 0.01).reshape(-1, 1))[:, 1]
        rating = predict @ density[0] * 0.01 # 0,01 - это шаг гистограммы
    else:
        rating = data_y.values[0]
    player_rating[player] = rating

In [123]:
#np.save("dataset.npy", player_rating)
player_rating = np.load("dataset.npy", allow_pickle=True).tolist()

In [125]:
avg = np.mean(list(player_rating.values()))

In [133]:
df['player_rating'] = df['player'].apply(lambda x: player_rating[x])

In [134]:
df.head()

,player,difficult,answer,question_id,player_rating
0,6212,0.116883,1,4772_0,0.707254
1,6212,0.220779,1,4772_1,0.707254
2,6212,0.554113,1,4772_2,0.707254
3,6212,0.480519,1,4772_3,0.707254
4,6212,0.121212,1,4772_4,0.707254


In [131]:
df.to_csv("data.csv", index=False)

In [160]:
ordered = dict(sorted(player_rating.items(), key=lambda item: -item[1]))

def id2name(idx):
    return players[idx]["name"] + " " + players[idx]["surname"]

for i in range(50):
    print(id2name(list(ordered.keys())[i]))

София Лебедева
Давид Кан
Михаил Завьялов
София Савенко
Алексей Антонов
Александр Корнюков
Полина Джегур
Елизавета Коваленко
Арина Гринко
Юлия Крюкова
Наталья Артемьева
Екатерина Горелова
Глеб Гаврилов
Семён Зайдельман
Дамир Тужушев
Алибек Аубакиров
Никита Романов
Александр Полторак
Никита Панфилов
Вячеслав Маслянко
Вадим Солодовников
Артём Захаров
Оксана Черенкова
Андрей Козенко
Дмитрий Вальтер
Валентина Подюкова
Елизавета Шкеда
Лидия Подшивайлова
Оганес Саакян
Сабит Мамбетов
Семён Кохоновер
Ольга Кинзерская
Илья Микулин
Василий Погребной
Константин Каволин
Мария Маторная
Елена Бровченко
Иван Катруха
Дмитрий Кудинов
Светлана Обогрелова
Максим Руссо
Данута Дембовская
Максим Ребров
Лейсан Ибнеева
Михаил Вантеев
Елизавета Корначёва
Александр Ковалевич
Константин Шаталин
Егор Лобынцев
Серафим Рылов


## 3 часть

In [81]:
def team_power(team):
    mates = [mate["player"]["id"] for mate in team]
    result = 1
    for mate in mates:
        sub = avg
        if mate in player_rating.keys():
            sub = player_rating[mate]
        result = result * (1 - sub)
    result = 1 - result
    return result

In [147]:
from scipy.stats import spearmanr, kendalltau


spearman = []
kendall = []
for tournament in test_tournaments:
    actual = []
    predicted = []
    for team in results[tournament]:
        if 'position' in team and \
        'teamMembers' in team:
            actual.append(1./team['position'])
            predicted.append(team_power(team['teamMembers']))
    spearman_coef = spearmanr(actual, predicted)[0]
    kendall_coef = kendalltau(actual, predicted)[0]
    
    if not np.isnan(spearman_coef):
        spearman.append(spearman_coef)
    if not np.isnan(kendall_coef):
        kendall.append(kendall_coef)
        
print("spearman", np.mean(spearman))
print("kendall", np.mean(kendall))


spearman 0.6704974265515671
kendall 0.5169115681490986


## 4 часть